### Sentencepiece : NSMC 데이터로 tokenizer 구성해보기
: https://github.com/google/sentencepiece  

내부 단어 분리를 위한 유용한 패키지: 구글의 센텐스피스(Sentencepiece)

In [1]:
! pip install sentencepiece

     ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/977.5 kB ? eta -:--:--
     -- ------------------------------------ 61.4/977.5 kB 1.1 MB/s eta 0:00:01
     ------ ------------------------------- 163.8/977.5 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 977.5/977.5 kB 7.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import sentencepiece as spm
import csv

In [6]:
# 데이터 읽어오기 : ratings_train.txt
naver_df = pd.read_table('./data/ratings_train.txt')
print(len(naver_df))

naver_df[:5]

150000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [8]:
# Null 값 제거 / 확인
naver_df = naver_df.dropna(how='any') # NULL 값 제거
print(naver_df.isnull().values.any()) # NULL 값이 없음을 확인


False


In [9]:
# 최종적으로 전처리된 텍스트를 'naver_review.txt'에 저장 (문장 \n 구분)
with open('./data/naver_review.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(naver_df['document']))

f.close()


In [11]:
# 학습하기: 2개 파일 생성 .vocab (subword), .model

spm.SentencePieceTrainer.Train('--input=./data/naver_review.txt, --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [13]:
# vocab 파일 확인해보기
vocab_list = pd.read_csv("./data/naver.vocab", sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,ᄏᄏ,-6


In [14]:
vocab_list.sample(10)

,0,1
2075,상을,-2072
4200,숭,-4197
1073,▁제발,-1070
2425,▁끊,-2422
2237,▁귀신,-2234
2499,▁아니면,-2496
390,시간,-387
1891,▁전설,-1888
1453,▁허무,-1450
183,▁눈,-180


In [16]:
# 모델 로드
sp = spm.SentencePieceProcessor()
vocab_file = './data/naver.model'
sp.load(vocab_file)

True

In [18]:
# 모델을 통해 tokenizing

lines = [
    "아 정말 피곤하다",
    "아 집 가고 싶다",
    "오늘은 집에 가고파 ㅋㅋㅋㅋ",
    "응애"
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

아 정말 피곤하다
['▁아', '▁정말', '▁피', '곤', '하다']
[7, 43, 493, 3852, 79]

아 집 가고 싶다
['▁아', '▁집', '▁가', '고', '▁싶다']
[7, 467, 46, 3293, 763]

오늘은 집에 가고파 ㅋㅋㅋㅋ
['▁오늘', '은', '▁집', '에', '▁가', '고', '파', '▁ᄏᄏᄏᄏ']
[960, 3310, 467, 3301, 46, 3293, 3578, 564]

응애
['▁응', '애']
[2247, 3452]



In [19]:
# 모델로 voca size 확인하기
sp.GetPieceSize()

5000

In [21]:
# subword to IT
sp.PieceToId('▁아')

7

In [22]:
# ID to subword
sp.IdToPiece(7)

'▁아'

In [24]:
# ids to subwords
sp.DecodeIds([7, 43, 493, 3852, 79])

'아 정말 피곤하다'

In [25]:
# subwords to 원형
sp.DecodePieces(['▁아', '▁집', '▁가', '고', '▁싶다'])

'아 집 가고 싶다'

In [26]:
# encode 메소드 중 out_type 인자를 통해: subwords, ids로 변환

print(sp.encode('와우 정말 대단한 영화입니다 ㅎㅎ', out_type=str))
print(sp.encode('와우 정말 대단한 영화입니다 ㅎㅎ', out_type=int))

['▁와우', '▁정말', '▁대단한', '▁영화입니다', '▁ᄒᄒ']
[3274, 43, 1695, 820, 434]
